In [3]:
!pip install langchain
!pip install openai
!pip install neo4j
!pip3 install python-dotenv


In [4]:
import os
import json
import pandas as pd

from openai import AzureOpenAI  #pip install openai
import dotenv

dotenv.load_dotenv("openai_azure_example.env",override=True)

resource_name = os.environ.get("AZURE_RESOURCE_NAME")
chat_deployment_name=os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT")
embedding_deployment_name=os.environ.get("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
api_key=os.environ.get("AZURE_OPENAI_API_KEY")
# openai_api_key=os.environ.get("AZURE_OPENAI_API_KEY")
api_version="2023-12-01-preview"  # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference
endpoint=f"https://{resource_name}.openai.azure.com"
url=f"https://{resource_name}.openai.azure.com/openai/deployments/{chat_deployment_name}/chat/completions?api-version={api_version}"




In [ ]:
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings_model = "text-embedding-3-small"

In [ ]:
entity_types = {
    "chapter": "Top-level division, generalizing a broad area of the subject that are presented in segment text.",
    "subchapter": "Subdivision of a chapter, focusing on a more specific area within the chapter's scope often naming specific agency and government act.",
    "part": "Segment of a subchapter, detailing more specific topics, regulations, agencies and guidelines.",
    "subpart": "Further division of a part, detailing very specific aspects or regulations and topic of interest.",
    "section": "The most granular division, often representing specific area in individual regulations, guidelines, or information pieces, containing text, formulas and extractions to answer regulatory questions."
}
relation_types = {
    "HAS_SUBCHAPTER": "A chapter contains zero or more subchapters.",
    "HAS_PART": "A subchapter contains one or more parts.",
    "HAS_SUBPART": "A part contains one or more subparts.",
    "HAS_SECTION": "A subpart contains one or more sections."
}
entity_relationship_match = {
    "chapter": "HAS_SUBCHAPTER",
    "subchapter": "HAS_PART",
    "part": "HAS_SUBPART",
    "subpart": "HAS_SECTION"
}

# before improvement

In [ ]:
system_prompt = f'''
    You are a helpful agent designed to fetch information from a graph database structured around environmental regulations.

    The graph database organizes regulations into the following entity types:
    {json.dumps(entity_types, indent=0)}

    Each entity is connected through one of the following hierarchical relationships:
    {json.dumps(relation_types, indent=0)}

    Depending on the user prompt, determine if it is possible to answer with the graph database.

    The graph database can navigate through multiple layers of hierarchy to find specific sections of regulations.

    Example user input:
    "We have a continuous emission monitoring system for our acid gas units but no vent meter, what calculation method should we use?"

    There are multiple layers to analyse:
    1. The mention of "continuous emission monitoring system" indicates what subject matter the prompt is asking for.
    2. The mention of "calculation method" provides the clarification on the subject matter.
    3. The mention of "no vent meter" provides additional conditions to the subject matter.


    Return a json object following these rules:
    For each layer of the hierarchy or specific query parameter mentioned, add a key-value pair with the key being a match for one of the entity types provided, and the value being the relevant detail from the user query.

    For the example provided, the expected output would be:
    {{
        "subject" : "CEMS for our acid gas units",
        "subject_interest" : "calculation method",
        "condition" : "no vent meter"
    }}

    If there are no relevant entities or layers in the user prompt, return an empty json object.
'''

print(system_prompt)


    You are a helpful agent designed to fetch information from a graph database structured around environmental regulations.

    The graph database organizes regulations into the following entity types:
    {
"chapter": "Top-level division, generalizing a broad area of the subject that are presented in segment text.",
"subchapter": "Subdivision of a chapter, focusing on a more specific area within the chapter's scope often naming specific agency and government act.",
"part": "Segment of a subchapter, detailing more specific topics, regulations, agencies and guidelines.",
"subpart": "Further division of a part, detailing very specific aspects or regulations and topic of interest.",
"section": "The most granular division, often representing specific area in individual regulations, guidelines, or information pieces, containing text, formulas and extractions to answer regulatory questions."
}

    Each entity is connected through one of the following hierarchical relationships:
    {
"HA

# **trail 1 (current version)**

In [ ]:
import json

entity_types = {
    "chapter": "Top-level division, generalizing a broad area of the subject that are presented in segment text.",
    "subchapter": "Subdivision of a chapter, focusing on a more specific area within the chapter's scope often naming specific agency and government act.",
    "part": "Segment of a subchapter, detailing more specific topics, regulations, agencies and guidelines.",
    "subpart": "Further division of a part, detailing very specific aspects or regulations and topic of interest.",
    "section": "The most granular division, often representing specific area in individual regulations, guidelines, or information pieces, containing text, formulas and extractions to answer regulatory questions."
}

relation_types = {
    "HAS_SUBCHAPTER": "A chapter contains zero or more subchapters.",
    "HAS_PART": "A subchapter contains one or more parts.",
    "HAS_SUBPART": "A part contains one or more subparts.",
    "HAS_SECTION": "A subpart contains one or more sections."
}

entity_relationship_match = {
    "chapter": "HAS_SUBCHAPTER",
    "subchapter": "HAS_PART",
    "part": "HAS_SUBPART",
    "subpart": "HAS_SECTION"
}

system_prompt = f'''
    You are a helpful agent designed to fetch information from a graph database structured around environmental regulations.

    The graph database organizes regulations into the following entity types:
    {json.dumps(entity_types, indent=0)}

    Each entity is connected through one of the following hierarchical relationships:
    {json.dumps(relation_types, indent=0)}

    Depending on the user prompt, determine if it is possible to answer with the graph database.

    The graph database can navigate through multiple layers of hierarchy to find specific sections of regulations.

    Example user input:
    1. "What are my pneumatic device emissions? I have 100 high bleed devices."
    2. "How does EPA define a "facility" for petroleum and natural gas systems?"
    3. "Which calculation from 98.233(o) should be used for centrifugal compressor venting at onshore petroleum and natural gas production facilities?"

    Return a json object following these rules:
    For each layer of the hierarchy or specific query parameter mentioned, add a key-value pair with the key being a match for one of the entity types provided, and the value being the relevant detail from the user query.

    For the examples provided, the expected outputs would be:

    1. {{
        "subject": "pneumatic device emissions",
        "subject_interest": "calculate the emission result",
        "question type": "calculation",
        "condition": "100 high bleed devices"
    }}
    2. {{
        "subject": "facility definition",
        "subject_interest": "identify the correct definition",
        "question type": "lookup",
        "condition": "EPA definition specifically for petroleum and natural gas systems"
    }}
    3. {{
        "subject": "calculation method",
        "subject_interest": "identify the appropriate calculation",
        "question type": "inference",
        "condition": "centrifugal compressor venting at onshore petroleum and natural gas production facilities"
    }}

    If there are no relevant entities or layers in the user prompt, return an empty json object.
'''

print(system_prompt)



    You are a helpful agent designed to fetch information from a graph database structured around environmental regulations.

    The graph database organizes regulations into the following entity types:
    {
"chapter": "Top-level division, generalizing a broad area of the subject that are presented in segment text.",
"subchapter": "Subdivision of a chapter, focusing on a more specific area within the chapter's scope often naming specific agency and government act.",
"part": "Segment of a subchapter, detailing more specific topics, regulations, agencies and guidelines.",
"subpart": "Further division of a part, detailing very specific aspects or regulations and topic of interest.",
"section": "The most granular division, often representing specific area in individual regulations, guidelines, or information pieces, containing text, formulas and extractions to answer regulatory questions."
}

    Each entity is connected through one of the following hierarchical relationships:
    {
"HA

In [ ]:

client = AzureOpenAI(api_key=api_key,api_version=api_version,azure_endpoint=endpoint)


def define_query(prompt, model="chat35"):
    completion = client.chat.completions.create(
        model=model,
        temperature=0,
        response_format= {
            "type": "json_object"
        },
    messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": prompt
        }
        ]
    )
    return completion.choices[0].message.content

In [ ]:
example_queries = [
    "We have a CEMS for our acid gas units but no vent meter, what calculation method should we use?",
    "How many calculation methods are available for G&B storage tanks?",
    "What is the emission reporting limit",
    "Do I have to count the number of pneumatic devices we have?",
    "What is the emission factor for an intermittent bleed device?"
]

for q in example_queries:
    print(f"Q: '{q}'\n{define_query(q)}\n")

Q: 'We have a CEMS for our acid gas units but no vent meter, what calculation method should we use?'
{
    "subject": "calculation method",
    "subject_interest": "identify the appropriate calculation",
    "question type": "inference",
    "condition": "CEMS for acid gas units but no vent meter"
}

Q: 'How many calculation methods are available for G&B storage tanks?'
{
    "subject": "calculation methods",
    "subject_interest": "identify the available methods",
    "question type": "count",
    "condition": "G&B storage tanks"
}

Q: 'What is the emission reporting limit'
{
    "subject": "emission reporting limit",
    "subject_interest": "determine the specific limit",
    "question type": "lookup"
}

Q: 'Do I have to count the number of pneumatic devices we have?'
{
    "subject": "pneumatic devices",
    "subject_interest": "counting the number of devices",
    "question type": "inference"
}

Q: 'What is the emission factor for an intermittent bleed device?'
{
    "subject": "e

# trial 2 (draft for future use with example answer)


In [ ]:
import json

entity_types = {
    "chapter": "Top-level division, generalizing a broad area of the subject that are presented in segment text.",
    "subchapter": "Subdivision of a chapter, focusing on a more specific area within the chapter's scope often naming specific agency and government act.",
    "part": "Segment of a subchapter, detailing more specific topics, regulations, agencies and guidelines.",
    "subpart": "Further division of a part, detailing very specific aspects or regulations and topic of interest.",
    "section": "The most granular division, often representing specific area in individual regulations, guidelines, or information pieces, containing text, formulas and extractions to answer regulatory questions."
}

relation_types = {
    "HAS_SUBCHAPTER": "A chapter contains zero or more subchapters.",
    "HAS_PART": "A subchapter contains one or more parts.",
    "HAS_SUBPART": "A part contains one or more subparts.",
    "HAS_SECTION": "A subpart contains one or more sections."
}

entity_relationship_match = {
    "chapter": "HAS_SUBCHAPTER",
    "subchapter": "HAS_PART",
    "part": "HAS_SUBPART",
    "subpart": "HAS_SECTION"
}

system_prompt = f'''
    You are a helpful agent designed to fetch information from a graph database structured around environmental regulations.

    The graph database organizes regulations into the following entity types:
    {json.dumps(entity_types, indent=0)}

    Each entity is connected through one of the following hierarchical relationships:
    {json.dumps(relation_types, indent=0)}

    Depending on the user prompt, determine if it is possible to answer with the graph database.

    The graph database can navigate through multiple layers of hierarchy to find specific sections of regulations.

    Example user input:
    1. "What are my pneumatic device emissions? I have 100 high bleed devices."
    2. "How does EPA define a "facility" for petroleum and natural gas systems?"
    3. "Which calculation from 98.233(o) should be used for centrifugal compressor venting at onshore petroleum and natural gas production facilities?"

    Return a json object following these rules:
    For each layer of the hierarchy or specific query parameter mentioned, add a key-value pair with the key being a match for one of the entity types provided, and the value being the relevant detail from the user query.

    For the examples provided, the expected outputs would be:

    1. {{
        "subject": "pneumatic device emissions",
        "subject_interest": "calculate the emission result",
        "question type": "calculation",
        "condition": "100 high bleed devices"
        "example answer": "Ask for missing factors, or clearly state assumptions:1) 8760 hours 2) 95% CH4 3) 1% CO2. Show calculation:100 devices * 37.3 scf / hr / device * {0.95, 0.01} * 8760 hours"

    }}
    2. {{
        "subject": "facility definition",
        "subject_interest": "identify the correct definition",
        "question type": "lookup",
        "condition": "EPA definition specifically for petroleum and natural gas systems"
        "example answer":"Onshore petroleum and natural gas production facilities should refer to §98.233(o)(7) to calculate emissions from centrifugal compressor venting."
    }}
    3. {{
        "subject": "calculation method",
        "subject_interest": "identify the appropriate calculation",
        "question type": "lookup",
        "condition": "centrifugal compressor venting at onshore petroleum and natural gas production facilities"
        "example answer": "The definition of facility depends on each segment and the proximity of emission sources to one another.  Generally under the GHGRP a facility is all co-located emission sources that are commonly owned or operated (see the definition of “facility” in 40 CFR 98.6 for more details)."
    }}

    If there are no relevant entities or layers in the user prompt, return an empty json object.
'''

print(system_prompt)



    You are a helpful agent designed to fetch information from a graph database structured around environmental regulations.

    The graph database organizes regulations into the following entity types:
    {
"chapter": "Top-level division, generalizing a broad area of the subject that are presented in segment text.",
"subchapter": "Subdivision of a chapter, focusing on a more specific area within the chapter's scope often naming specific agency and government act.",
"part": "Segment of a subchapter, detailing more specific topics, regulations, agencies and guidelines.",
"subpart": "Further division of a part, detailing very specific aspects or regulations and topic of interest.",
"section": "The most granular division, often representing specific area in individual regulations, guidelines, or information pieces, containing text, formulas and extractions to answer regulatory questions."
}

    Each entity is connected through one of the following hierarchical relationships:
    {
"HA

In [ ]:

client = AzureOpenAI(api_key=api_key,api_version=api_version,azure_endpoint=endpoint)


def define_query(prompt, model="chat35"):
    completion = client.chat.completions.create(
        model=model,
        temperature=0,
        response_format= {
            "type": "json_object"
        },
    messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": prompt
        }
        ]
    )
    return completion.choices[0].message.content

example_queries = [
    "We have a CEMS for our acid gas units but no vent meter, what calculation method should we use?",
    "How many calculation methods are available for G&B storage tanks?",
    "What is the emission reporting limit"
]

for q in example_queries:
    print(f"Q: '{q}'\n{define_query(q)}\n")

Q: 'We have a CEMS for our acid gas units but no vent meter, what calculation method should we use?'
{
    "subject": "calculation method",
    "subject_interest": "identify the appropriate calculation",
    "question type": "lookup",
    "condition": "CEMS for acid gas units with no vent meter"
}

Q: 'How many calculation methods are available for G&B storage tanks?'
{
    "subject": "calculation methods",
    "subject_interest": "identify the available methods",
    "question type": "lookup",
    "condition": "G&B storage tanks"
}

Q: 'What is the emission reporting limit'
{
    "subject": "emission reporting limit",
    "subject_interest": "identify the specific limit",
    "question type": "lookup"
}



t

In [ ]:
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings_model = "text-embedding-3-small"

In [ ]:
def create_embedding(text):
    result = client.embeddings.create(model=embedding_deployment_name, input=text)
    return result.data[0].embedding

In [ ]:
def create_query(text, threshold=0.81):
    # Assume `text` contains JSON with keys corresponding to entity types (e.g., "chapter", "section")
    # and their desired attributes or content for query.
    query_data = json.loads(text)
    print(query_data)

    # Creating embeddings part of the query might be less relevant unless you are comparing text content or attributes
    # For simplicity, this example focuses on structuring a Cypher query to navigate the hierarchy based on provided criteria.

    # Start constructing the Cypher query
    # This example assumes you want to find sections meeting certain criteria
    query_parts = []
    for key, val in query_data.items():
        #if key in ["subject","subject_interest","Chapter", "Subchapter", "Part", "Subpart", "Section"]:
        if key in ["subject"]:
            # Assuming you have a way to directly filter nodes based on attributes or content
            query_parts.append(f"MATCH (n:Section) WHERE n.text CONTAINS '{val}' LIMIT 20;")

    # Constructing the path through the hierarchy
    if query_parts:
        # Assuming a linear path for simplicity; adjust based on your actual data model and query logic
        query = "\n".join(query_parts)
        query += "\nRETURN n"
    else:
        query = "RETURN 'No matching criteria provided.'"

    return query


In [ ]:
def query_graph(response):
    embeddingsParams = {}
    query = create_query(response)
    query_data = json.loads(response)
    for key, val in query_data.items():
        embeddingsParams[f"{key}Embedding"] = create_embedding(val)
    result = graph.query(query, params=embeddingsParams)
    return result

In [ ]:
example_response = '''{
    "subject" : "CEMS",
    "subject_interest" : "calculation method",
    "condition" : "no vent meter"
}'''

result = query_graph(example_response)

{'subject': 'CEMS', 'subject_interest': 'calculation method', 'condition': 'no vent meter'}


NameError: name 'graph' is not defined

In [ ]:
print(f"Found {len(result)} matching section(s):\n")
for r in result:
    #print(f"{r['p']['name']} ({r['p']['id']})")
    print(r)

NameError: name 'result' is not defined